In [37]:
import pandas as pd
import os
from processMain import processMain
import keras

In [38]:
file_path = '../data/home-credit-credit-risk-model-stability/csv_files/test'

# open test_base.csv
# open test_appleprev_1_0.csv

df_base = pd.read_csv(file_path + '/test_base.csv')
df_appleprev_1_0 = pd.read_csv(file_path + '/test_applprev_1_0.csv')
df_person_1 = pd.read_csv(file_path + '/test_person_1.csv')
df_person_2 = pd.read_csv(file_path + '/test_person_2.csv')
df = pd.merge(df_base, df_appleprev_1_0, on='case_id', how='left')
df = pd.merge(df, df_person_1, on='case_id', how='left')
df = pd.merge(df, df_person_2, on='case_id', how='left')


df = processMain(df, keep_col=['case_id'])

# Now concat my test with my train to get the right columns
df_train = pd.read_csv("../data/mergedDatasets/person&Applprev.csv")

df_final = pd.read_csv('../data/processed/final.csv')
missing_columns = list(set(df_final.columns) - set(df.columns))

print(f'len of columns: {len(df.columns)}')
print(f'len of final training col: {len(df_final.columns)}')
print(f'Len of missing: {len(missing_columns)}')

# display columsn that are in df but not in df_final
unexpected_columns = list(set(df.columns) - set(df_final.columns))
print(f'len of unexpected: {unexpected_columns}')

# remove unexpected columns form df
df = df.drop(unexpected_columns, axis=1)
print(f'target in df: {"target" in df.columns}')

default_values = pd.DataFrame(0, index=df.index, columns=missing_columns)
df = pd.concat([df, default_values], axis=1)
df = df.drop('target', axis=1)
print(f'len of columns after adding missing: {len(df.columns)}')

df.to_csv('../data/processed/test.csv', index=False)


# # Calculate missing columns
# missing_columns = list(set(df_final.columns) - set(df.columns))
# # Calculate unexpected columns
# unexpected_columns = list(set(df.columns) - set(df_final.columns))
# # Display the results
# print("Number of missing columns:", len(missing_columns))
# print("Missing columns:", missing_columns)
# print("Number of unexpected columns:", len(unexpected_columns))
# print("Unexpected columns:", unexpected_columns)



# Now add min missing columns (fill with 0's)
# print(f'are columns equal: {df_final.columns == df.columns}')
# print(f'columns of final: {df_final.columns}')
# print(f'columsn of test: {df.columns}')


df columns in process Main: Index(['case_id', 'district_544M', 'mainoccupationinc_437A',
       'cancelreason_3545846M', 'annuity_853A', 'credamount_590A'],
      dtype='object')
len of columns: 19
len of final training col: 456
Len of missing: 442
len of unexpected: ['mainoccupationinc_437A_0.0', 'case_id', 'cancelreason_3545846M_0', 'mainoccupationinc_437A_200000.0', 'district_544M_0']
target in df: False
len of columns after adding missing: 455


In [39]:
# Now Load the model in and make predictions
model = keras.models.load_model('../model/model.keras')

# make predictions
predictions = model.predict(df)
print(predictions)

df = pd.read_csv('../data/home-credit-credit-risk-model-stability/csv_files/test/test_base.csv')
df = df[['case_id']]
df['score'] = predictions
df.to_csv('../data/predictions.csv', index=False)

1/1 [==============================] - 0s 30ms/step
[[0.17570569]
 [0.19442187]
 [0.11452612]
 [0.09791911]
 [0.5506507 ]
 [0.5506507 ]
 [0.5506507 ]
 [0.2315045 ]
 [0.19276224]
 [0.5506507 ]]
